See here for instructions to set up jupyter lab:
https://stackoverflow.com/questions/51922480/javascript-error-ipython-is-not-defined-in-jupyterlab

In [1]:
import os
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cymr import models
from psifr import fr
from cfr import task
data_file = '/Users/morton/PycharmProjects/cmr_cfr/data/MortEtal13.csv'
sem_file = '/Users/morton/PycharmProjects/cmr_cfr/data/cfr_wikiw2v.mat'
fit_dir = '/Users/morton/Dropbox/work/cfr/test4'
if not os.path.exists(fit_dir):
    os.makedirs(fit_dir)
param_file = '/Users/morton/PycharmProjects/cmr_cfr/fits/test/param_2020-04-25T01:02:03.csv'

In [2]:
results = pd.read_csv(param_file, index_col=0)
subj_param = results.loc[1]
subj_param

B_rec      0.706104
Afc        0.000000
Dfc        4.591694
Acf        6.084015
Dcf        7.822870
Lfc        2.964409
Lcf        1.829305
P1         7.444525
P2         3.486755
T          1.999996
X1         0.007863
X2         0.262239
B_enc      0.810710
logl    -901.204434
Name: 1, dtype: float64

In [7]:
data = pd.read_csv(data_file)
list_data = data.loc[(data['subject'] == 1) & (data['list'] == 2)].copy()
list_data

,subject,list,position,trial_type,item,item_number,session,list_type,category,response,response_time,list_category
43,1,2,1,study,SEAN PENN,229,1,mixed,cel,3.0,1.255,mixed
44,1,2,2,study,AUDREY HEPBURN,20,1,mixed,cel,3.0,1.040,mixed
45,1,2,3,study,ST PATRICKS CATHEDRAL,464,1,mixed,loc,2.0,1.164,mixed
46,1,2,4,study,LES INVALIDES,365,1,mixed,loc,2.0,0.829,mixed
47,1,2,5,study,GREAT ZIMBABWE RUINS,341,1,mixed,loc,3.0,0.872,mixed
48,1,2,6,study,BRYCE CANYON,282,1,mixed,loc,2.0,1.241,mixed
49,1,2,7,study,TROPHY,746,1,mixed,obj,3.0,0.862,mixed
50,1,2,8,study,BRUSH,549,1,mixed,obj,3.0,0.779,mixed
51,1,2,9,study,SUPREME COURT,470,1,mixed,loc,2.0,1.245,mixed
52,1,2,10,study,SAN FRANCISCO,445,1,mixed,loc,4.0,0.854,mixed


In [22]:
list_data['item_index'] = list_data['position'] - 1
cats = list_data['category'].unique()
study_data = list_data.loc[list_data['trial_type'] == 'study']
cat_patterns = np.zeros((study_data.shape[0], len(cats)), dtype=int)
for i, cat in enumerate(cats):
    cat_patterns[study_data['category'] == cat, i] = 1

patterns = {'vector': {'loc': np.eye(24, dtype=int), 'cat': cat_patterns}}
weights = {'fcf': {'loc': 'w_loc', 'cat': 'w_cat'}}
subj_param['w_loc'] = 1
subj_param['w_cat'] = 2

In [25]:
model = models.CMRDistributed()
state_list = model.record_network(list_data, subj_param, 
                                  patterns=patterns, weights=weights)

In [26]:
%matplotlib widget
from cymr import network
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display

# initialize the plot
fig, ax = network.init_plot()
state = state_list[0]
state[0].plot(ax=ax)

def update(n=1):
    """Update network data on reach time step."""
    print(n)
    ax['c'].images[0].set_data(state[n].c[None, :])
    ax['c_in'].images[0].set_data(state[n].c_in[None, :])
    ax['f'].images[0].set_data(state[n].f[None, :])
    f_in = state[n].f_in
    ax['f_in'].images[0].set_data(f_in[None, :])
    ax['f_in'].images[0].set_clim(np.min(f_in), np.max(f_in))
    ax['w_fc_exp'].images[0].set_data(state[n].w_fc_exp)
    ax['w_cf_exp'].images[0].set_data(state[n].w_cf_exp)

w = interactive(update, n=widgets.IntSlider(min=0, max=len(state) - 1, step=1, value=0))
display(w)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=0, description='n', max=40), Output()), _dom_classes=('widget-interact',…